# Korea Imports - Analysis

## Dependencies

In [1]:
import os
import sys
import duckdb

sys.path.append("../ingest")
import config # type: ignore

## Check for Files in Clean Data Directory

In [2]:
clean_data = "." + config.Local_Dir.Korea_Imports["clean"]
print(os.listdir(clean_data))

['clean_koreaimports.parquet']


## Duckdb Relation

In [3]:
conn = duckdb.connect()
rel = conn.from_parquet(clean_data + config.datasets["korea-imports"]["filename-clean"])

## Dataset Summary

In [4]:
summary = rel.query(virtual_table_name="summary", sql_query="SUMMARIZE rel")
summary

┌──────────────┬─────────────┬───────────┬───────────────┬───────────────┬────────────────────┬────────────────────┬───────────────────┬──────────────────┬────────────────────┬───────┬─────────────────┐
│ column_name  │ column_type │    min    │      max      │ approx_unique │        avg         │        std         │        q25        │       q50        │        q75         │ count │ null_percentage │
│   varchar    │   varchar   │  varchar  │    varchar    │     int64     │      varchar       │      varchar       │      varchar      │     varchar      │      varchar       │ int64 │  decimal(9,2)   │
├──────────────┼─────────────┼───────────┼───────────────┼───────────────┼────────────────────┼────────────────────┼───────────────────┼──────────────────┼────────────────────┼───────┼─────────────────┤
│ partnerDesc  │ VARCHAR     │ Argentina │ World         │             5 │ NULL               │ NULL               │ NULL              │ NULL             │ NULL               │  5483 │    

## Top Korea Imports from World by Value (USD)

- MINERAL FUELS, MINERAL OILS AND PRODUCTS OF THEIR DISTILLATION; BITUMINOUS SUBSTANCES; MINERAL WAXES = **27**.
- ELECTRICAL MACHINERY AND EQUIPMENT AND PARTS THEREOF; SOUND RECORDERS AND REPRODUCERS; TELEVISION IMAGE AND SOUND RECORDERS AND REPRODUCERS, PARTS AND ACCESSORIES OF SUCH ARTICLES = **85**.
- NUCLEAR REACTORS, BOILERS, MACHINERY AND MECHANICAL APPLIANCES; PARTS THEREOF = **84**.
- VEHICLES; OTHER THAN RAILWAY OR TRAMWAY ROLLING STOCK, AND PARTS AND ACCESSORIES THEREOF = **87**.
- INORGANIC CHEMICALS; ORGANIC AND INORGANIC COMPOUNDS OF PRECIOUS METALS; OF RARE EARTH METALS, OF RADIO-ACTIVE ELEMENTS AND OF ISOTOPES = **28**.

In [5]:
top_world_imports = rel.query(
    virtual_table_name="top_world_imports",
    sql_query="""SELECT cmdCode, SUM(primaryValue) totalValue
    FROM rel
    WHERE partnerDesc = 'World'
    GROUP BY cmdCode
    ORDER BY totalValue desc
    LIMIT 10;"""
)
top_world_imports

┌─────────┬────────────────┐
│ cmdCode │   totalValue   │
│  uint8  │     double     │
├─────────┼────────────────┤
│      27 │ 171354533642.0 │
│      85 │ 119478238633.0 │
│      84 │  63072733926.0 │
│      87 │  22134904666.0 │
│      90 │  21958896343.0 │
│      28 │  21516316779.0 │
│      26 │  19707452376.0 │
│      72 │  16471724419.0 │
│      29 │  13572871176.0 │
│      39 │  12710272397.0 │
├─────────┴────────────────┤
│ 10 rows        2 columns │
└──────────────────────────┘

## Top Korea Import from China and USA by Value (USD)

- ELECTRICAL MACHINERY AND EQUIPMENT AND PARTS THEREOF; SOUND RECORDERS AND REPRODUCERS; TELEVISION IMAGE AND SOUND RECORDERS AND REPRODUCERS, PARTS AND ACCESSORIES OF SUCH ARTICLES = **85**.
- MINERAL FUELS, MINERAL OILS AND PRODUCTS OF THEIR DISTILLATION; BITUMINOUS SUBSTANCES; MINERAL WAXES = **27**.
- INORGANIC CHEMICALS; ORGANIC AND INORGANIC COMPOUNDS OF PRECIOUS METALS; OF RARE EARTH METALS, OF RADIO-ACTIVE ELEMENTS AND OF ISOTOPES = **28**.
- NUCLEAR REACTORS, BOILERS, MACHINERY AND MECHANICAL APPLIANCES; PARTS THEREOF = **84**.
- VEHICLES; OTHER THAN RAILWAY OR TRAMWAY ROLLING STOCK, AND PARTS AND ACCESSORIES THEREOF = **87**.

In [6]:
top_partners_imports = rel.query(
    virtual_table_name="top_partners_imports",
    sql_query="""SELECT partnerDesc, cmdCode, SUM(primaryValue) totalValue
    FROM rel
    WHERE partnerDesc IN ('China', 'USA')
    GROUP BY partnerDesc, cmdCode
    ORDER BY totalValue desc
    LIMIT 10;"""
)
top_partners_imports

┌─────────────┬─────────┬───────────────┐
│ partnerDesc │ cmdCode │  totalValue   │
│   varchar   │  uint8  │    double     │
├─────────────┼─────────┼───────────────┤
│ China       │      85 │ 50431712097.0 │
│ USA         │      27 │ 22310370371.0 │
│ China       │      84 │ 16001128335.0 │
│ China       │      28 │ 13632775411.0 │
│ USA         │      84 │  9605944597.0 │
│ China       │      72 │  6350946542.0 │
│ USA         │      85 │  6077486751.0 │
│ China       │      29 │  4924468291.0 │
│ China       │      39 │  4670551997.0 │
│ USA         │      90 │  4632022958.0 │
├─────────────┴─────────┴───────────────┤
│ 10 rows                     3 columns │
└───────────────────────────────────────┘

## Top Korea Imports form Iberoamerican Partners by Value (USD)

- MINERAL FUELS, MINERAL OILS AND PRODUCTS OF THEIR DISTILLATION; BITUMINOUS SUBSTANCES; MINERAL WAXES = **27**.

In [7]:
top_iberoamerican_imports = rel.query(
    virtual_table_name="top_iberoamerican_imports",
    sql_query="""SELECT partnerDesc, cmdCode, SUM(primaryValue) totalValue
    FROM rel
    WHERE partnerDesc NOT IN ('World', 'China', 'USA', 'Colombia')
    GROUP BY partnerDesc, cmdCode
    ORDER BY totalValue desc
    LIMIT 10;"""
)
top_iberoamerican_imports

┌─────────────┬─────────┬──────────────┐
│ partnerDesc │ cmdCode │  totalValue  │
│   varchar   │  uint8  │    double    │
├─────────────┼─────────┼──────────────┤
│ Chile       │      28 │ 3180601162.0 │
│ Mexico      │      27 │ 2412138484.0 │
│ Chile       │      26 │ 1800014774.0 │
│ Chile       │      74 │ 1640027047.0 │
│ Brazil      │      27 │ 1577571834.0 │
│ Mexico      │      26 │ 1319968529.0 │
│ Brazil      │      26 │ 1119738049.0 │
│ Mexico      │      84 │ 1099731904.0 │
│ Brazil      │      10 │ 1074432811.0 │
│ Argentina   │      10 │  905523992.0 │
├─────────────┴─────────┴──────────────┤
│ 10 rows                    3 columns │
└──────────────────────────────────────┘

## Top Korea Imports from Colombia by Value (USD)

In [8]:
top_colombia_imports = rel.query(
    virtual_table_name="top_colombia_imports",
    sql_query="""SELECT partnerDesc, cmdCode, SUM(primaryValue) totalValue
    FROM rel
    WHERE partnerDesc = 'Colombia'
    GROUP BY partnerDesc, cmdCode
    ORDER BY totalValue desc
    LIMIT 10;"""
)
top_colombia_imports

┌─────────────┬─────────┬─────────────┐
│ partnerDesc │ cmdCode │ totalValue  │
│   varchar   │  uint8  │   double    │
├─────────────┼─────────┼─────────────┤
│ Colombia    │      27 │ 736307073.0 │
│ Colombia    │       9 │ 153016607.0 │
│ Colombia    │      72 │  52079540.0 │
│ Colombia    │      74 │  33488166.0 │
│ Colombia    │       6 │  28706954.0 │
│ Colombia    │      21 │  13848453.0 │
│ Colombia    │      38 │  12558231.0 │
│ Colombia    │       8 │   6878028.0 │
│ Colombia    │      26 │   5236512.0 │
│ Colombia    │      76 │   5006008.0 │
├─────────────┴─────────┴─────────────┤
│ 10 rows                   3 columns │
└─────────────────────────────────────┘